# Creating a conversation AI chatbot

- using encoder-decoder framework
- This bot answers question given in full dataset in kaggle challenge

# 1)- Importing key modules

In [1]:
# support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 2)-Loading Dataset

In [3]:
print(os.listdir('raw_data'))

['ai.yml', 'botprofile.yml', 'computers.yml', 'emotion.yml', 'food.yml', 'gossip.yml', 'greetings.yml', 'health.yml', 'history.yml', 'humor.yml', 'literature.yml', 'money.yml', 'movies.yml', 'politics.yml', 'psychology.yml', 'science.yml', 'sports.yml', 'trivia.yml']


In [4]:
import os
import yaml

dir_path = 'raw_data'
files_list = os.listdir(dir_path + os.sep)

In [5]:
files_list

['ai.yml',
 'botprofile.yml',
 'computers.yml',
 'emotion.yml',
 'food.yml',
 'gossip.yml',
 'greetings.yml',
 'health.yml',
 'history.yml',
 'humor.yml',
 'literature.yml',
 'money.yml',
 'movies.yml',
 'politics.yml',
 'psychology.yml',
 'science.yml',
 'sports.yml',
 'trivia.yml']

### reading tags in data

In [6]:
for filepath in files_list:
    stream = open(os.path.join(dir_path, filepath), 'rb')
    docs = yaml.safe_load(stream)
print(docs)

{'categories': ['trivia'], 'conversations': [['Who was the 37th President of the United States?', 'Richard Nixon'], ['What year was President John F. Kennedy assassinated?', '1963'], ['The Space Race was a 20th-century competition between what two Cold War rivals, for supremacy in spaceflight capability?', 'The Soviet Union and the United States.'], ['What was the name of the first artificial Earth satellite?', 'Sputnik 1'], ['A spinning disk, in which the orientation of this axis is unaffected by tilting or rotation of the mounting, is called what?', 'A gyroscope.'], ['The Hubble Space Telescope, launched into low Earth orbit in 1990, is named after what American astronomer?', 'Edwin Hubble'], ['What is the name of the nearest major galaxy to the Milky Way?', 'The Andromeda Galaxy.'], ['God Save the Queen is the national anthem of what country?', 'The United Kingdom of Great Britain'], ['The Celtic Shelf, the seabed under the Celtic Sea is a part of the continental shelf of what conti

In [7]:
docs["categories"]

['trivia']

In [8]:
docs["conversations"]

[['Who was the 37th President of the United States?', 'Richard Nixon'],
 ['What year was President John F. Kennedy assassinated?', '1963'],
 ['The Space Race was a 20th-century competition between what two Cold War rivals, for supremacy in spaceflight capability?',
  'The Soviet Union and the United States.'],
 ['What was the name of the first artificial Earth satellite?', 'Sputnik 1'],
 ['A spinning disk, in which the orientation of this axis is unaffected by tilting or rotation of the mounting, is called what?',
  'A gyroscope.'],
 ['The Hubble Space Telescope, launched into low Earth orbit in 1990, is named after what American astronomer?',
  'Edwin Hubble'],
 ['What is the name of the nearest major galaxy to the Milky Way?',
  'The Andromeda Galaxy.'],
 ['God Save the Queen is the national anthem of what country?',
  'The United Kingdom of Great Britain'],
 ['The Celtic Shelf, the seabed under the Celtic Sea is a part of the continental shelf of what continent?',
  'Europe'],
 ['Do

In [9]:
con_access=docs["conversations"]
type(con_access)

list

In [10]:
con_access

[['Who was the 37th President of the United States?', 'Richard Nixon'],
 ['What year was President John F. Kennedy assassinated?', '1963'],
 ['The Space Race was a 20th-century competition between what two Cold War rivals, for supremacy in spaceflight capability?',
  'The Soviet Union and the United States.'],
 ['What was the name of the first artificial Earth satellite?', 'Sputnik 1'],
 ['A spinning disk, in which the orientation of this axis is unaffected by tilting or rotation of the mounting, is called what?',
  'A gyroscope.'],
 ['The Hubble Space Telescope, launched into low Earth orbit in 1990, is named after what American astronomer?',
  'Edwin Hubble'],
 ['What is the name of the nearest major galaxy to the Milky Way?',
  'The Andromeda Galaxy.'],
 ['God Save the Queen is the national anthem of what country?',
  'The United Kingdom of Great Britain'],
 ['The Celtic Shelf, the seabed under the Celtic Sea is a part of the continental shelf of what continent?',
  'Europe'],
 ['Do

**This is only one of many data file**

### Separating Questions and answers

In [11]:
questions = list()
answers = list()
for con in con_access:
    if len(con)>2:
        questions.append(con[0])
        replies=con[1:]
        ans=""
        for rep in replies:
            ans+=' '+rep
            answers.append(ans)
    elif len(con)>1:
        questions.append(con[0])
        answers.append(con[1])

In [12]:
questions

['Who was the 37th President of the United States?',
 'What year was President John F. Kennedy assassinated?',
 'The Space Race was a 20th-century competition between what two Cold War rivals, for supremacy in spaceflight capability?',
 'What was the name of the first artificial Earth satellite?',
 'A spinning disk, in which the orientation of this axis is unaffected by tilting or rotation of the mounting, is called what?',
 'The Hubble Space Telescope, launched into low Earth orbit in 1990, is named after what American astronomer?',
 'What is the name of the nearest major galaxy to the Milky Way?',
 'God Save the Queen is the national anthem of what country?',
 'The Celtic Shelf, the seabed under the Celtic Sea is a part of the continental shelf of what continent?',
 'Dolphins use a sense, similar to sonar, to determine the location and shape of nearby items.']

In [13]:
answers

['Richard Nixon',
 '1963',
 'The Soviet Union and the United States.',
 'Sputnik 1',
 'A gyroscope.',
 'Edwin Hubble',
 'The Andromeda Galaxy.',
 'The United Kingdom of Great Britain',
 'Europe',
 'Echolocation']

### for more than one yml.file

for all files in our raw_data folder

In [14]:
full_docs=[]

for filepath in files_list:
    stream = open(os.path.join(dir_path, filepath), 'rb')
    docs = yaml.safe_load(stream)
    full_docs.append(docs)

In [15]:
full_docs[12]

{'categories': ['movies'],
 'conversations': [['you sound like hal', "to me that's a great compliment."],
  ['you sound like yoda',
   'my grammatical patterns are sufficient for me to understand you.'],
  ['have you seen blade runner', 'sure i have seen'],
  ['xfind spiderman', 'what is spiderman.'],
  ['when did teknolust', 'teknolust was released in 2002.'],
  ['what is spiderman', 'a comic book story made into a movie.'],
  ['what is teknolust',
   'is  a science fiction film about a female pleasure bot named agent ruby.'],
  ['what is solaris', 'solaris is an edition of unix from sun.'],
  ['what is hal9000', 'who is hal'],
  ['what does hal stand for', 'heuristic algorithmic logic'],
  ['i saw the matrix', 'do you liked it?'],
  ['is hal 9000 your boyfriend', 'no.'],
  ['is hal safe', 'not for humans'],
  ['is hal nice', 'only to other robots.'],
  ['is hal alive', 'he is a fictional character.'],
  ['is hal dead', 'he is a fictional robot.'],
  ['is hal', 'hal has a few issues t

### All in one

In [16]:
questions = list()
answers = list()
for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

In [17]:
questions[0]

'What is AI?'

In [18]:
answers[0]

'Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.'

### Add start and end tag with answers

In [19]:
answers_with_tags=list()
for i in range( len(answers)):
    if type(answers[i])==str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers=list()
for ans in answers_with_tags:
    answers.append('<START> '+ans+' <END>')

In [20]:
questions[0]

'What is AI?'

In [21]:
answers[0]

'<START> Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think. <END>'

# 3)- Preprocessing Data

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, preprocessing, utils
import yaml

print( tf.version)

<module 'tensorflow_core._api.v2.version' from 'C:\\Users\\hassan.sherwani\\Miniconda3\\envs\\tensorflow\\lib\\site-packages\\tensorflow_core\\_api\\v2\\version\\__init__.py'>


In [23]:
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions+answers)
VOCAB_SIZE= len(tokenizer.word_index)+1
print('VOCAB SIZE : {}'.format(VOCAB_SIZE))

VOCAB SIZE : 1894


### 3.1.tokenizer

In [24]:
from gensim.models import Word2Vec
import re

vocab=[]
for word in tokenizer.word_index:
    vocab.append(word)
    
def tokenize(sentences):
    tokens_list=[]
    vocabulary=[]
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '^a-zA-Z',' ',sentence)
        tokens = sentence.split()
        vocabulary+=tokens
        tokens_list.append(tokens)
    return tokens_list,vocabulary

p=tokenize(questions+answers)
model = Word2Vec(p[0])

#### 3.2.a.tokenized_questions

In [25]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
encoder_input_data = np.array( padded_questions )
print(encoder_input_data.shape, maxlen_questions)

(564, 22) 22


#### 3.2.b.tokenized_answers

In [26]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape, maxlen_answers)

(564, 74) 74


In [27]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i]=tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
one_hot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)
decoder_output_data = np.array(one_hot_answers)
print(decoder_output_data.shape)

(564, 74, 1894)


# 4)- Model Building

In [28]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    378800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    378800      input_2[0][0]                    
______________________________________________________________________________________________

### fit and save model

In [29]:
%%time
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save( 'model.h5' )

Train on 564 samples
Epoch 1/150
564/564 [==============================] - 13s 22ms/sample - loss: 1.4518 - accuracy: 0.1041
Epoch 2/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.2890 - accuracy: 0.0711
Epoch 3/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.1346 - accuracy: 0.0700
Epoch 4/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.1067 - accuracy: 0.0859
Epoch 5/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.0938 - accuracy: 0.1174
Epoch 6/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.0827 - accuracy: 0.0988
Epoch 7/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.0690 - accuracy: 0.1329
Epoch 8/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.0515 - accuracy: 0.1428
Epoch 9/150
564/564 [==============================] - 5s 9ms/sample - loss: 1.0349 - accuracy: 0.1518
Epoch 10/150
564/564 [============================

564/564 [==============================] - 6s 10ms/sample - loss: 0.4315 - accuracy: 0.5257
Epoch 80/150
564/564 [==============================] - 6s 10ms/sample - loss: 0.4258 - accuracy: 0.5297
Epoch 81/150
564/564 [==============================] - 6s 10ms/sample - loss: 0.4188 - accuracy: 0.5393
Epoch 82/150
564/564 [==============================] - 6s 11ms/sample - loss: 0.4119 - accuracy: 0.5447
Epoch 83/150
564/564 [==============================] - 6s 10ms/sample - loss: 0.4052 - accuracy: 0.5519
Epoch 84/150
564/564 [==============================] - 6s 10ms/sample - loss: 0.3983 - accuracy: 0.5597
Epoch 85/150
564/564 [==============================] - 6s 10ms/sample - loss: 0.3918 - accuracy: 0.5680
Epoch 86/150
564/564 [==============================] - 5s 10ms/sample - loss: 0.3850 - accuracy: 0.5777
Epoch 87/150
564/564 [==============================] - 5s 10ms/sample - loss: 0.3785 - accuracy: 0.5845
Epoch 88/150
564/564 [==============================] - 6s 10ms/samp

**Accuracy value started from 11% and eventually improved with epochs upto value of 87.9 %**

### inference

During the inference phase, a trained Seq2Seq model gets a source sentence. It uses this to obtain an encoder_state which is used to initialize the decoder. The conversational process starts as soon as the decoder receives a special symbol, <start>, denoting the start of the decoding process.

In [30]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [31]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

# 5)-Evaluation

In [33]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_response = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_response += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_response.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_response )

Enter question : hi
 hello end
Enter question : exit


KeyError: 'exit'

any context that is not trained,  will not be entertained.

# END of NOTEBOOK